In [14]:
# importing the needed libraries
import requests 
import json
import pymysql
import warnings
import codecs
from bs4 import BeautifulSoup
from collections import namedtuple
import pandas as pd
import time
import re
import numpy as np

In [15]:
def main():

    url = 'https://www.ebay.com/sch/i.html?_nkw=ipad+4&LH_BIN=1&_ipg=100' 
    user_agent = {'User-agent': 'Mozilla/5.0'} 


    
    response = requests.get(url, headers = user_agent)
    #write the htm file for each page
    with open('ebay_ipad4_01.htm','w') as file:
        file.write(response.text)
        
    for pgn in range(2, 11):
        time.sleep(5) 
        url_pgn = url + '&_pgn=' + str(pgn) 
        filename = 'ebay_ipad4_' + str(pgn).zfill(2) + '.htm'
        response = requests.get(url_pgn, headers = user_agent)
        with open(filename, 'w') as file:
            file.write(response.text)
    
            
    #c
    #seperate items for sponsor and nonsponsor
    sponsor=[]
    nonsponsor=[]
    for pgn in range(1,11):
        filename = 'ebay_ipad4_' + str(pgn).zfill(2) + '.htm'
        with open(filename, 'r') as file:
            text = file.read()
            soup = BeautifulSoup(text, 'html.parser') 
            items = soup.select("li.s-item")
            for item in items:
                a = item.find("a")
                if not a:
                    continue
                
                text = item.find("span").get_text()
                if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):  
                    url_simplify = re.sub("(.*)\\?.*", r"\1\n", a['href'])           
                    #print(url_simplify)
                    sponsor.append(url_simplify)
                else:
                    url_simplify1 = re.sub("(.*)\\?.*", r"\1\n", a['href'])    
                    nonsponsor.append(url_simplify1)
    #write the separate item number into txt file
    with open("sponsored.txt","w") as file:
        for spon in np.unique(sponsor):
            file.writelines(spon)
    with open("non-sponsored.txt","w") as file:
        for nonspon in np.unique(nonsponsor):
            file.writelines(nonspon)
if __name__ == "__main__":
    main()

In [16]:
#create folders
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Example
createFolder('./sponsored/')
createFolder('./non-sponsored/')

In [ ]:
from multiprocessing import Pool
p = Pool(5)
def f(x):
    return x*x
p.map(f, [1,2,3])

In [17]:
#write a htm file for each item
def create():
    try:
        with open("sponsored.txt","r") as file:
            sponsortext=file.readlines()
        with open("non-sponsored.txt","r") as file:
            nonsponsortext=file.readlines()
        for s in sponsortext:
            num=re.sub("(.*)/([0-9]*)\n",r'\2',s)
            filename=num+'.htm'
            user_agent = {'User-agent': 'Mozilla/5.0'}
            url=re.sub("(.*)\n",r'\1',s)
            response = requests.get(url, headers = user_agent)    
            with open(os.path.join('./sponsored/',filename), 'w') as file:
                file.write(response.text)
        for s in nonsponsortext:
            num=re.sub("(.*)/([0-9]*)\n",r'\2',s)
            filename=num+'.htm'

            user_agent = {'User-agent': 'Mozilla/5.0'} 
            url=re.sub("(.*)\n",r'\1',s)
            response = requests.get(url, headers = user_agent)    
            with open(os.path.join('./non-sponsored/',filename), 'w') as file:
                file.write(response.text)
        time.sleep(2) 
    except IOError as e:
        pass
        print(e)
#Make sure to catch an error and continue if your query runs into problems connecting to eBay 
#(e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [74]:
#select the object we want to analyze
def selector(filename):
    with open(filename, 'r') as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser')
    #print('filename')
    #item_id
    num=re.sub('./(.*)/(.*).htm',r'\2',filename)
    num=codecs.escape_encode(str.encode(num))[0].decode()



    ##seller name
    seller=soup.find_all("span",class_='mbg-nw') #selector command
    seller_name=re.sub("(.*)>(.*)<(.*)",r'\2',str(seller))
    #print("seller name: "+seller_name)

    #seller score
    score=soup.find_all('span',class_='mbg-l') #selector command

    s_score=re.sub("(.*)\n(.*)>([0-9]*)</a>(.*)\n(.*)",r'\3',str(score))
    #print(s_score)
    if s_score=='[]':
        seller_score=None
    else:
        seller_score=str(float(s_score))
        seller_score=codecs.escape_encode(str.encode(seller_score))[0].decode()
    #print("seller score: "+ str(seller_score))

    #item price
    if soup.find_all(id='convbinPrice'): #CAD selecting
        price=soup.find_all(id='convbinPrice')
        item_price=re.sub("(.*)>US (.*)<(.*)",r'\2',str(price))
        item_price=re.sub("(.*)<span>(.*)",r'\1',item_price)
        item_p2=re.sub("^(.)",r'',item_price)
        #print(item_p2)
    else:
        price=soup.find_all(id='prcIsum')#selector command  #USD selecting
        if len(price)==0:
            price=soup.find_all(id="prcIsum_bidPrice")


        item_price=re.sub("(.*)>(.*)<(.*)",r'\2',str(price))
        item_price=re.sub("(.*)>\n(.*)<(.*)",r'\2',item_price)
        item_price=re.sub("(.*)/ea",r'\1',item_price)
        item_p=re.sub("(.* )(.*)",r'\2',item_price)
        item_p=re.sub("^(.)",r'',item_p)
        item_p2=re.sub(",",r'',item_p)
        #print(item_p)

    if item_p2==']':
        item_p1=None
    else: 
        item_p1=str(float(item_p2)*100)
        item_p1=codecs.escape_encode(str.encode(item_p1))[0].decode()
    #print("item price: "+str(item_p1))


    #item sold
    sold=soup.find_all("a",class_="vi-txt-underline")#selector command      
    item_sold=re.sub("(.*)>([0-9]*)(.*)<(.*)",r'\2',str(sold))
    if item_sold=='[]':
        item_sold1=None
    else:
        item_sold1=str(item_sold)
        item_sold1=codecs.escape_encode(str.encode(item_sold1))[0].decode()
    #print("item sold: " + str(item_sold1))

    #best offer available
    offer=soup.find_all('div',class_='vi-bbox-dspn u-flL lable boLable')
    o=re.search("Best Offer",str(offer))
    if o:
        bestoffer='1'

    else:
        bestoffer='0'
    bestoffer=codecs.escape_encode(str.encode(bestoffer))[0].decode()
    #print("best offer available: " +bestoffer)

    #title
    title=soup.find_all(id='itemTitle')#selector command
    titlename=re.sub("(.*)>(.*)<(/h1>])",r'\2',str(title))
    titlename=codecs.escape_encode(str.encode(titlename))[0].decode()
    #print("title: " + titlename)

    #returns allowed
    returned=soup.find_all(id='vi-ret-accrd-txt')#selector command
    returnallow=re.sub("(.*)>(.*)<(.*)",r'\2',str(returned))
    #print(returnallow)
    if re.search("not",returnallow):
        returnallow='1'

    else:
        returnallow='0'

    returnallow=codecs.escape_encode(str.encode(returnallow))[0].decode()
    #print(returnallow)
    #print("return: " + returnallow)

    #shipping price
    shipping=soup.find_all(id='fshippingCost')#selector command
    if len(shipping)>0:
        shippingprice=re.sub("(.*)>\n(.*)>(.*)<(.*)\n<(.*)",r'\3',str(shipping))
        if shippingprice=='FREE':
            shippingprice='0'
            shippingprice=codecs.escape_encode(str.encode(shippingprice))[0].decode()
        elif len(shippingprice)<10:
            shippingprice=re.sub("^(.)",r'',shippingprice)
            shippingprice=str(float(shippingprice)*100)
            shippingprice=codecs.escape_encode(str.encode(shippingprice))[0].decode()
        else:
            shippingprice=None
    else:
        shippingprice=None
    #print("shipping price: "+ shippingprice)

    #condition
    condition=soup.find(id='vi-itm-cond')
    condition_regex=re.sub('(.*)>(.*)<(.*)',r'\2',str(condition))
    condition_regex=codecs.escape_encode(str.encode(condition_regex))[0].decode()
    #print("condition: "+ condition_regex)
    #print('\n')
    
    return(num,seller_name,seller_score,item_p1,item_sold1,bestoffer,titlename,returnallow,shippingprice,condition_regex)

In [75]:
#test the selector function 
filename='./non-sponsored/111121377380.htm'
print(selector(filename))

('111121377380', 'eshilacoota', '1739.0', '84889.0', '6', '0', 'Apple iPad 4th Generation with Retina Display 32GB, Wi-Fi 9.7in - White...', '0', '0', 'New')


In [83]:
#create database
import pymysql
import warnings
import requests
import json
import codecs
from bs4 import BeautifulSoup
import stat


#ignore warnings
warnings.filterwarnings("ignore")
SQL_DB = "eBay"
#TERM = "hella"

def main():
    
    SQL_TABLE_ebay = "eBay_items"
    SQL_TABLE_ebay_DEF = "(" + \
            "item_id VARCHAR(255) NOT NULL PRIMARY KEY" + \
            ",sponsor VARCHAR(50)" + \
            ",seller_name VARCHAR(255)" + \
            ",seller_score INTEGER(20)" + \
            ",item_price INTEGER(10)" + \
            ",item_sold INTEGER(10)" + \
            ",best_offer_available INTEGER(2)" + \
            ",title VARCHAR(255)" + \
            ",return_allowed INTEGER(2)" + \
            ",shipping_price INTEGER(10)" + \
            ",condition_1 VARCHAR(255)" + \
            ")"
    create_sql_table(SQL_TABLE_ebay, SQL_TABLE_ebay_DEF)
    
    try:
        #connect to server
        psw=''
        conn = pymysql.connect(host='localhost', user = 'root', password = psw, database = 'eBay')
        cursor = conn.cursor()
        cursor.execute("set sql_mode = ''")

        
        with open("sponsored.txt","r") as file:
            sponsortext=file.readlines()
        with open("non-sponsored.txt","r") as file:
            nonsponsortext=file.readlines()
        #insert sponsor items into the database
        for s in np.unique(sponsortext):
            num_1=re.sub("(.*)/([0-9]*)\n",r'\2',s)
            print(num_1)

            filename='./sponsored/'+num_1+'.htm'
            selector(filename)
            result=selector(filename)
#            print(result)
            #print(num)

            prepared_stmt = "INSERT INTO " + SQL_TABLE_ebay + " (item_id,sponsor, seller_name,seller_score,item_price,item_sold,best_offer_available,title,return_allowed,shipping_price,condition_1) values ('{}','{}', '{}','{}', '{}','{}', '{}','{}', '{}','{}', '{}');"
            stmt = prepared_stmt.format(result[0],'sponsor',result[1],result[2],result[3],result[4],result[5],result[6],result[7],result[8],result[9])
            cursor.execute(stmt)
            conn.commit()
        #insert nonsponsor items into the database
        for j in np.unique(nonsponsortext):
            num_2=re.sub("(.*)/([0-9]*)\n",r'\2',j)
            print(num_2)

            filename='./non-sponsored/'+num_2+'.htm'
            selector(filename)
            result=selector(filename)
#            print(result)
            #print(num)
            prepared_stmt = "INSERT IGNORE INTO " + SQL_TABLE_ebay + " (item_id,sponsor, seller_name,seller_score,item_price,item_sold,best_offer_available,title,return_allowed,shipping_price,condition_1) values ('{}','{}', '{}','{}', '{}','{}', '{}','{}', '{}','{}', '{}');"
            stmt = prepared_stmt.format(result[0],'nonsponsor',result[1],result[2],result[3],result[4],result[5],result[6],result[7],result[8],result[9])
            cursor.execute(stmt)
            conn.commit()
            

        time.sleep(0.5)
        cursor.close()
        conn.close()

    except IOError as e:
        pass
        print(e)

  #create sql table 
def create_sql_table(SQL_TABLE_ebay, SQL_TABLE_ebay_DEF):
    try:
        
        #connect to server
        psw=''
        conn = pymysql.connect(host='localhost', user = 'root', password = psw)
        cursor = conn.cursor()

        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE_ebay + " " + SQL_TABLE_ebay_DEF + ";";
        print(query)
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)
        pass



if __name__ == "__main__":
    main()


CREATE DATABASE IF NOT EXISTS eBay
CREATE TABLE IF NOT EXISTS eBay.eBay_items5 (item_id VARCHAR(255) NOT NULL PRIMARY KEY,sponsor VARCHAR(50),seller_name VARCHAR(255),seller_score INTEGER(20),item_price INTEGER(10),item_sold INTEGER(10),best_offer_available INTEGER(2),title VARCHAR(255),return_allowed INTEGER(2),shipping_price INTEGER(10),condition_1 VARCHAR(255));
273164342327
202878382158
174152002678
174176985396
192827303952
181656129916
181637088535
401634536254
183890928232
362784041660
371755393228
303405584026
293383390892
293452172205
283745160382
153686003469
173611128685
173609728979
171822367762
171763041177
202065893518
181723497677
171869058779
171763035773
182662075533
182386213481
171807143092
181809799186
201394204476
273069634007
183382559195
183192071315
182734350511
182584748144
202036657907
172839316201
202036617347
133307538348
164036071867
143504391488
143504482729
143504482766
181950392757
202066332518
173506928462
172887900328
223739694932
323972923975
18178330

124059914288
164063398453
274244585280
324059681910
153809047441
333491370258
401992776063
143504434458
143504391518
164040981595
143504482823
164040981415
164040981540
143510109249
164035871782
143504434375
164040981555
143504482898
164040981456
133310924492
164035871874
233471703220
174153185873
174156356764
174174359686
174175356235
124072099042
174172961287
174180018835
174152438629
174157242707
174157245263
174160053760
174160056879
174163839958
174163841743
174169087289
174170131531
174170137159
174181390423
264614009013
324065459732
252822791253
253015370112
262976839396
111352455994
142867976011
121773773325
333396274922
141219049671
121432759747
302990137862
201698546991
202859243001
181917778829
171933419906
171919500509
124073460833
123964091486
172065171183
173475681087
172032974899
323996401117
401831405956
164023319335
402070685563
312898782230
113995466399
124073023317
201494493469
333506030463
143495768317
182522405070
173521485824
153701470440
113868230659
164008007458

In [84]:
# select summary statistics from the database
def summary():
    #connect to server
    psw=''
    conn = pymysql.connect(host='localhost', user = 'root', password = psw, database = 'eBay')
    cursor = conn.cursor()
    sql = "SELECT sponsor,\
                condition_1,\
                SUM(seller_score) AS sum_seller_score,\
                AVG(seller_score) AS mean_seller_score,\
                MIN(seller_score) AS minimum_seller_score,\
                MAX(seller_score) AS maximum_seller_score,\
                STD(seller_score) AS sd_seller_score,\
                SUM(item_price) AS sum_item_price,\
                AVG(item_price) AS mean_item_price,\
                MIN(item_price) AS minimum_item_price,\
                MAX(item_price) AS maximum_item_price,\
                STD(item_price) AS sd_item_price,\
                SUM(best_offer_available) AS sum_best_offer_available,\
                AVG(best_offer_available) AS mean_best_offer_available,\
                MIN(best_offer_available) AS minimum_best_offer_available,\
                MAX(best_offer_available) AS maximum_best_offer_available,\
                STD(best_offer_available) AS sd_best_offer_available,\
                SUM(return_allowed) AS sum_return_allowed,\
                AVG(return_allowed) AS mean_return_allowed,\
                MIN(return_allowed) AS minimum_return_allowed,\
                MAX(return_allowed) AS maximum_return_allowed,\
                STD(return_allowed) AS sd_best_return_allowed,\
                SUM(shipping_price) AS sum_shipping_price,\
                AVG(shipping_price) AS mean_shipping_price,\
                MIN(shipping_price) AS minimum_shipping_price,\
                MAX(shipping_price) AS maximum_shipping_price,\
                STD(shipping_price) AS sd_shipping_price\
                FROM eBay.eBay_items\
                GROUP BY sponsor,condition_1\
                ORDER BY sponsor;"
    cursor.execute(sql);
    data=cursor.fetchall()
    #result=pd.DataFrame(records)
    
    columnDes = cursor.description #get the description information
    columnNames = [columnDes[i][0] for i in range(len(columnDes))]
    df = pd.DataFrame([list(i) for i in data],columns=columnNames)
    print(df)


  
    conn.commit()
    cursor.close()
    conn.close()

In [85]:
summary()

      sponsor               condition_1 sum_seller_score mean_seller_score  \
0  nonsponsor  Manufacturer refurbished       8940897914     58057778.6623   
1  nonsponsor                       New      13529904274    121891029.4955   
2  nonsponsor                  Open box      19580942545    222510710.7386   
3  nonsponsor        Seller refurbished      65023463066    344039487.1217   
4  nonsponsor                      Used      83975532232    189989891.9276   
5     sponsor  Manufacturer refurbished          5384842        84138.1563   
6     sponsor                  Open box           169486        84743.0000   
7     sponsor        Seller refurbished       8590583153    715881929.4167   
8     sponsor                      Used      13943771023   1161980918.5833   

   minimum_seller_score  maximum_seller_score  sd_seller_score sum_item_price  \
0                     0            2147483647     3.413286e+08        3216827   
1                     0            2147483647     4.84430

## Difference on sponsored items and nonsponsored items
#### 1. all the sellers of the sponsored items have seller score, but the minimum of nonsponsor is normally zero
#### 2. all the sponosored items have minimum item price
#### 3. non-sponsored items have a higher return allow rate 
#### 4. sponsored items mostly have free shipping, non-sponsored items have higher shipping price